## Using the model
Once the prediction has been achived with the previously targeted data [**supervised learning**] and all the pre-processing of the data and the model have been stored, the same model is ready to be used with un-targeted data and obtain **predictions & probabilities** about new employees. 

In [1]:
from absenteeism_module import *

In [2]:
model = absenteeism_model("model_mod", "scaler")
model.load_and_clean_data("...\Absenteeism_new_data.csv")
new_predictions = model.predicted_outputs()
new_predictions.drop(["Body Mass Index"], axis = 1, inplace = True) #column number incongruency with SQL if we dont drop BMI
new_predictions.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Education,Children,Pet,Probability,Prediction
0,0,0.0,0,1,6,179,30,1,0,0,0.122799,0
1,1,0.0,0,0,6,361,28,0,1,4,0.873392,1
2,0,0.0,0,1,6,155,34,0,2,0,0.268305,0
3,0,0.0,0,1,6,179,40,1,2,0,0.196385,0
4,1,0.0,0,0,6,155,34,0,2,0,0.723502,1


## Conencting Python & SQL
SQL stores and organizes the information, both for targeted data used for the model construction and the new data, with the predictions obtained with the model.
Python allow us to clean, process and work with data. [** SO WE NEED TO CONNECT THESE TWO TOOLS IN ORDER TO WORK IN A MORE EFFICIENT WAY**]


In [3]:
import pymysql
conn = pymysql.connect(database = "predicted_outputs", user = "****", password = "********")
cursor = conn.cursor()

In [4]:
insert = "INSERT INTO predicted_outputs VALUES "

for i in range(new_predictions.shape[0]): #generating a new line of SQL code fow each df_row
    insert += "("
    for j in range(new_predictions.shape[1]): #extracting the value of each parameter for SQL, in each df_column
        insert += str(new_predictions[new_predictions.columns.values[j]][i]) + ", "
    insert = insert[:-2] + "), " #when value extration is over in df_row end SQL code with ),
insert = insert[:-2] + ";" #when value extraction is over in df end SQL query with ;    

In [5]:
cursor.execute(insert)
conn.commit()